In [ ]:
import functools
import pathlib

import lifetimes

In [ ]:
path = pathlib.Path("/home/fabian/Downloads")
level = 500
outfile = path / f"pressure_level_{level}_daily_mean_2017_2020.nc"
paths = lifetimes.utils.list_files(path, pattern="pl_*.nc")
paths

In [ ]:
preprocess = functools.partial(
    lifetimes.features.methods.select_level_and_calculate_daily_mean,
    level=level,
)
ds = lifetimes.features.EcmwfIfsHresDataset(
    paths=paths,
    overlapping=True,
    preprocessing=preprocess,
)

In [ ]:
ds.to_netcdf(outfile, drop_variables=["t"])